In [24]:
import os
import numpy as np
import pandas as pd
import holidays

# Introduction

This notebook was used to generate the external_data.csv file which was sent to the RAMP platform with the python code. In order to accept the join, the final csv must include 3 merging keys : 

- DateOfDeparture : a date column indicating the day of departure of the flight.
- Departure : the airport code of the airport where the flight departed.
- Arrival : the airport code of the airport where the flight arrived.

# Preprocessing

The training and testing data are located in the folder `data`. They are compressed `csv` file (i.e. `csv.bz2`). We can load the dataset using pandas.

In [25]:
data1 = pd.read_csv(
    os.path.join('data', 'train.csv.bz2')
)
data2 = pd.read_csv(
    os.path.join('data', 'test.csv.bz2')
) 

data = pd.concat([data1, data2])

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128 entries, 0 to 2225
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DateOfDeparture   11128 non-null  object 
 1   Departure         11128 non-null  object 
 2   Arrival           11128 non-null  object 
 3   WeeksToDeparture  11128 non-null  float64
 4   log_PAX           11128 non-null  float64
 5   std_wtd           11128 non-null  float64
dtypes: float64(3), object(3)
memory usage: 608.6+ KB


,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159


In [26]:
date_col = "DateOfDeparture"

## Utilities

The following helper functions are used during the ETL process.

In [27]:
def date_encoding(df, date_col, date_keys=['year', 'month', 'day', 'weekday', 'week'],
 time_elts=['year', 'month', 'day', 'weekday', 'week']):

    """
    Takes a date column and parses it in order to create new columns. The parameter date_keys is a list used to determine which columns must be created, while the parameter time_elts is used to determine the name of the said columns. The rename parameter is used to rename the date column to the convention "DateOfDeparture" which is used in every dataset that we use.
    """

    df_encoded = df.copy()
    df_encoded.loc[:, date_col] = pd.to_datetime(df_encoded[date_col])

    if 'year' in date_keys:
        df_encoded.loc[:, time_elts[0]] = df_encoded[date_col].dt.year
    if 'month' in date_keys:
        df_encoded.loc[:, time_elts[1]] = df_encoded[date_col].dt.month
    if 'day' in date_keys:
        df_encoded.loc[:, time_elts[2]] = df_encoded[date_col].dt.day
    if 'weekday' in date_keys:
        df_encoded.loc[:, time_elts[3]] = df_encoded[date_col].dt.weekday
    if 'week' in date_keys:
        df_encoded.loc[:, time_elts[4]] = df_encoded[date_col].dt.week

    return df_encoded

def city_name_encoding(df):

    """
    Loads a reference dataset which joins the name of a city on its airport code. The operation is both performed on the arrival and on the departure columns.
    """

    df_codes = pd.read_csv(os.path.join('data', 'airports.csv'))
    df_codes = df_codes.loc[~df_codes["iata_code"].isnull(), ["iata_code", "municipality"]]
    df_codes.loc[:, "municipality"] = df_codes.loc[:, "municipality"
    ].replace({"Dallas-Fort Worth":"Dallas", "Newark":"New York"})
    df_codes = df_codes.rename(columns={"iata_code":"Departure", "municipality":"Departure City"})
    
    df_encoded = df.merge(df_codes, how="left", on="Departure")
    df_codes = df_codes.rename(columns={"Departure":"Arrival", "Departure City":"Arrival City"})
    df_encoded = df_encoded.merge(df_codes, how="left", on="Arrival")
    return df_encoded

def purchase_date(df, date_col):

    """
    Uses the "WeeksToDeparture" column to create a new column called "Purchase Date" in order to know when the customer bought his flight ticket. This function is useful for columns which data depends on the purchase date rather than on the flight date, such as the Google Trends one. It also uses the date_encoding function to create date columns which will be used to join on the correct dates.
    """

    df = df.copy()
    time_diff = (df[date_col] - pd.to_datetime("1970-01-01") ).dt.days - 7 * df["WeeksToDeparture"]
    df["Purchase Date"] = pd.to_datetime("1970-01-01") + time_diff.apply(np.ceil).apply(lambda x: pd.Timedelta(x, unit='D'))
    df = date_encoding(df, date_col, date_keys=['year', 'month', 'weekday', 'week'],
        time_elts=['p_year', 'p_month', 'p_day', 'p_weekday', 'p_week'], rename=False)
    return df

def financial_df_importer(company, date_col="DateOfDeparture"):

    """
    Imports a specified Yahoo stocks dataset.
    """

    df = pd.read_csv(
    os.path.join('data', f'yahoo-{company}.csv'), sep=",",
    parse_dates=["Date"]
    )
    stock_name = company.capitalize()
    df = df.rename(columns={"Close": stock_name, "Date":date_col})
    
    df = df.loc[:, [date_col, stock_name]]

    return df

def financial_cleaner(df, stock_names, fill_info=True, erase_na=False):

    """
    Multiple operations are performed : first, the list of stocks given to the function is capitalized. Then, the user can choose to fill missing stock data (the stocks are not opened on week-ends, creating gaps in the dataset). 
    """

    stock_names = [stock_name.capitalize() for stock_name in stock_names]
    if fill_info:
        df.loc[:, stock_names] = df.loc[:, stock_names].fillna(method='pad')
    if erase_na and (fill_info==False):
        df = df.dropna()

    print(df.info())
    return df

def financial_pipeline(full_df, companies, purchase_date=True, date_col="DateOfDeparture"):

    """
    Handles the loading of all Yahoo datasets in a single dataset.
    """

    if purchase_date:
        merge_key = "Purchase Date"
    else:
        merge_key = date_col

    for company in companies:
        df_financial = financial_df_importer(company)

        if purchase_date:
            df_financial = df_financial.rename(columns={date_col:"Purchase Date"})

        full_df = full_df.merge(df_financial, how='left', on=[merge_key])
    
    full_df = financial_cleaner(full_df, companies, fill_info=True)

    return full_df


In [28]:
df = data.copy()
df = date_encoding(df, date_col)
df = city_name_encoding(df)
df = purchase_date(df, date_col)


<ipython-input-27-c525858f7127>:20: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_encoded.loc[:, time_elts[4]] = df_encoded[date_col].dt.week


In [29]:
df.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,year,month,day,weekday,week,Departure City,Arrival City,Purchase Date,p_year,p_month,p_weekday,p_week
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647,2012,6,19,1,25,Chicago,Dallas,2012-03-21,2012,6,1,25
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734,2012,9,10,0,37,Las Vegas,Denver,2012-06-03,2012,9,0,37
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883,2012,10,5,4,40,Denver,Los Angeles,2012-07-21,2012,10,4,40
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202,2011,10,9,6,40,Atlanta,Chicago,2011-07-21,2011,10,6,40
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159,2012,2,21,1,8,Denver,San Francisco,2011-12-03,2012,2,1,8


We now have a completed dataset with the names of the airport cities on departure and arrival (df).

In [30]:
print(df["Departure City"].sort_values().unique())
print(df["Arrival City"].sort_values().unique())

print(df[date_col].min())
print(df[date_col].max())


['Atlanta' 'Boston' 'Charlotte' 'Chicago' 'Dallas' 'Denver' 'Detroit'
 'Houston' 'Las Vegas' 'Los Angeles' 'Miami' 'Minneapolis' 'New York'
 'Orlando' 'Philadelphia' 'Phoenix' 'San Francisco' 'Seattle']
['Atlanta' 'Boston' 'Charlotte' 'Chicago' 'Dallas' 'Denver' 'Detroit'
 'Houston' 'Las Vegas' 'Los Angeles' 'Miami' 'Minneapolis' 'New York'
 'Orlando' 'Philadelphia' 'Phoenix' 'San Francisco' 'Seattle']
2011-09-01 00:00:00
2013-03-05 00:00:00


# External data merging

In this section, we will preprocess and join data on the initial dataset in order to export the external_data.csv file used in the prediction model.

## US Labor data

This dataset contains metrics on the US economy from the US Department of Labor.

In [31]:
df_labor = pd.read_csv(
    os.path.join('data', 'US Labor.csv'), sep=";"
)

df_labor["Price"] = df_labor["Price"].str.replace(",",".").astype("float64")

In [32]:
df_labor = df_labor.rename(columns={"year":"p_year", "month":"p_month"})
full_df = df.merge(df_labor, how='left', on=["p_year", "p_month"])
full_df.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,year,month,day,weekday,...,Purchase Date,p_year,p_month,p_weekday,p_week,Civ. Labor Force,Employment,CPI,CPI - Urban / clerks,Price
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647,2012,6,19,1,...,2012-03-21,2012,6,1,25,155083,142391,229478,226036,199.8
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734,2012,9,10,0,...,2012-06-03,2012,9,0,37,155160,143044,231407,228184,179.8
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883,2012,10,5,4,...,2012-07-21,2012,10,4,40,155554,143431,231317,227974,183.4
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202,2011,10,9,6,...,2011-07-21,2011,10,6,40,153961,140368,226421,223043,177.9
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159,2012,2,21,1,...,2011-12-03,2012,2,1,8,154671,141858,227663,224317,180.5


## Yahoo Finance 

The following datasets are coming from the Yahoo finance website. They all contain the price of a stock at closing time on a given day. Since they each added precision to the model, we built a helper function to easily process any new stock that we decided to add to the dataset.

### Booking.com stock performance

In [33]:
companies = ['booking', 'delta', 'aairlines']
full_df = financial_pipeline(full_df, companies, purchase_date=True)
full_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128 entries, 0 to 11127
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DateOfDeparture       11128 non-null  datetime64[ns]
 1   Departure             11128 non-null  object        
 2   Arrival               11128 non-null  object        
 3   WeeksToDeparture      11128 non-null  float64       
 4   log_PAX               11128 non-null  float64       
 5   std_wtd               11128 non-null  float64       
 6   year                  11128 non-null  int64         
 7   month                 11128 non-null  int64         
 8   day                   11128 non-null  int64         
 9   weekday               11128 non-null  int64         
 10  week                  11128 non-null  int64         
 11  Departure City        11128 non-null  object        
 12  Arrival City          11128 non-null  object        
 13  Purchase Date   

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,year,month,day,weekday,...,p_weekday,p_week,Civ. Labor Force,Employment,CPI,CPI - Urban / clerks,Price,Booking,Delta,Aairlines
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647,2012,6,19,1,...,1,25,155083,142391,229478,226036,199.8,703.650024,9.74,7.63
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734,2012,9,10,0,...,0,37,155160,143044,231407,228184,179.8,703.650024,9.74,7.63
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883,2012,10,5,4,...,4,40,155554,143431,231317,227974,183.4,703.650024,9.74,7.63
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202,2011,10,9,6,...,6,40,153961,140368,226421,223043,177.9,526.010010,8.17,6.90
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159,2012,2,21,1,...,1,8,154671,141858,227663,224317,180.5,526.010010,8.17,6.90


## Google Trends

Another helping dataset is the Google Trends one. It was obtained by entering the sentences "Flight airport X to airport Y" in Google Trends, in order to know the demand for a particular flight on a given day. The data is available on a weekly basis, and it was sometimes pulled twice a week, so we had to add more preprocessing in order to avoid duplicate entries.

In [34]:
col_names = ["year", "week", "Arrival City", "Frequency"]
df_google = pd.DataFrame({'year': pd.Series([], dtype='int64'),
                'week': pd.Series([], dtype='int64'),
                'Arrival City': pd.Series([], dtype='str'),
                'Frequency': pd.Series([], dtype='int64')})
root_name = 'multiTimeline_'

for k in range(0, 5):
    if k==0:
        target = root_name[:-1] + '.csv'
    else:
        target = root_name + str(k) + '.csv'
    df_temp = pd.read_csv(
        os.path.join('data', target),
        parse_dates=["Date"])
    df_temp["week"] = df_temp["Date"].dt.week
    df_temp["year"] = df_temp["Date"].dt.year
    df_temp = df_temp.drop(columns=["Date"])
    df_temp = df_temp.set_index(["year", "week"])
    df_temp = df_temp.stack()
    df_temp = df_temp.reset_index()
    df_temp.columns = col_names
    df_google = df_google.append(df_temp)


df_google = df_google.groupby(["year", "week", "Arrival City"]).mean()
df_google = df_google.reset_index()

full_df = full_df.merge(df_google, how='left', on=["year", "week", "Arrival City"])

print(len(full_df))
df_google.head(20)
full_df.info()

11128
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128 entries, 0 to 11127
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   DateOfDeparture       11128 non-null  datetime64[ns]
 1   Departure             11128 non-null  object        
 2   Arrival               11128 non-null  object        
 3   WeeksToDeparture      11128 non-null  float64       
 4   log_PAX               11128 non-null  float64       
 5   std_wtd               11128 non-null  float64       
 6   year                  11128 non-null  int64         
 7   month                 11128 non-null  int64         
 8   day                   11128 non-null  int64         
 9   weekday               11128 non-null  int64         
 10  week                  11128 non-null  int64         
 11  Departure City        11128 non-null  object        
 12  Arrival City          11128 non-null  object        
 13  Purchase D

<ipython-input-34-0c92e07a4035>:16: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_temp["week"] = df_temp["Date"].dt.week


In [35]:
full_df.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,year,month,day,weekday,...,p_week,Civ. Labor Force,Employment,CPI,CPI - Urban / clerks,Price,Booking,Delta,Aairlines,Frequency
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647,2012,6,19,1,...,25,155083,142391,229478,226036,199.8,703.650024,9.74,7.63,18.0
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734,2012,9,10,0,...,37,155160,143044,231407,228184,179.8,703.650024,9.74,7.63,14.0
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883,2012,10,5,4,...,40,155554,143431,231317,227974,183.4,703.650024,9.74,7.63,16.0
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202,2011,10,9,6,...,40,153961,140368,226421,223043,177.9,526.010010,8.17,6.90,24.0
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159,2012,2,21,1,...,8,154671,141858,227663,224317,180.5,526.010010,8.17,6.90,14.0


# Features engineering 

In this section, we used existing columns to create new features for the model.

## Holidays

In [36]:
us_holidays = holidays.US()
full_df["is_holiday"] = full_df["DateOfDeparture"].apply(lambda x: 1 if x in us_holidays else 0)
full_df[full_df["is_holiday"]==1].head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,year,month,day,weekday,...,Civ. Labor Force,Employment,CPI,CPI - Urban / clerks,Price,Booking,Delta,Aairlines,Frequency,is_holiday
24,2012-11-12,EWR,BOS,11.000000,11.740518,8.711759,2012,11,12,0,...,155338,143333,230221,226595,181.8,588.400024,9.06,11.04,16.0,1
54,2011-12-25,EWR,LAX,12.818182,10.007415,8.671993,2011,12,25,6,...,153995,140902,225672,222166,179.5,511.119995,8.14,6.11,23.0,1
68,2012-12-25,DEN,LAX,15.034483,10.391932,11.890821,2012,12,25,1,...,155628,143330,229601,225889,194.2,617.760010,9.28,11.62,23.0,1
77,2013-01-21,DEN,ORD,9.631579,10.935530,7.158604,2013,1,21,0,...,155763,143292,230280,226520,187.0,617.520020,9.49,12.06,32.0,1
119,2011-11-24,DEN,ORD,10.176471,8.285437,7.307832,2011,11,24,3,...,154128,140826,226230,222813,176.7,529.559998,8.39,5.74,22.0,1


## Distances

We used Haversine's formula in order to compute the distances between two airports.

The formula states that :

$d = R * c$

With :

$a = sin^2(\frac{\delta \phi}{2}) + cos(\phi_1) * cos(\phi_2) * sin^2(\frac{\delta \lambda}{2})$

And :

$c = 2 * atan2(\sqrt{a}, \sqrt{1-a})$ 



where :

- d is the distance between both points ;

- $\phi$ is the latitude of a point;

- $\lambda$ is the longitude of a point ;

- $R$ is the Earth's radius.

In [37]:
deg_to_rad = lambda x: x * np.pi/180

def haversine_dist(lat1,lng1,lat2,lng2):
    R = 6371 # Radius of the earth in km
    deg_lat = deg_to_rad(lat2-lat1)
    deg_lng = deg_to_rad(lng2-lng1) 
    a = np.sin(deg_lat/2) * np.sin(deg_lat/2) + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(deg_lng/2) * np.sin(deg_lng/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    d = R * c ## Distance in km
    return np.round(d, 3)

In [38]:
cities_list = df.copy()["Departure City"].unique()
df_geo = pd.DataFrame(cities_list, columns=["Departure City"])
df_geo["key"] = 1
df_geo = df_geo.merge(df_geo, on="key").drop(columns=["key"]).rename(
    columns={"Departure City_x":"Departure City", "Departure City_y":"Arrival City"})

In [39]:
data_cities = pd.read_csv('data/uscities.csv')

data_cities = data_cities[data_cities["city"].isin(list(cities_list))]
data_cities = data_cities.sort_values(by="population")
data_cities = data_cities.loc[data_cities.groupby("city")["population"].idxmax()]
data_cities = data_cities.loc[:, ["city", "lat", "lng", "population", "density"]].rename(columns={"city":"Departure City", "lat":"Dep_lat", "lng":"Dep_lng", "population":"Dep_population", "density":"Dep_density"})

data_cities.head()

,Departure City,Dep_lat,Dep_lng,Dep_population,Dep_density
7,Atlanta,33.7627,-84.4224,5449398,1441.0
9,Boston,42.3188,-71.0846,4688346,5532.0
37,Charlotte,35.2080,-80.8304,1512923,1113.0
2,Chicago,41.8373,-87.6862,8604203,4574.0
4,Dallas,32.7936,-96.7662,5743938,1526.0


In [40]:
df_geo = df_geo.merge(data_cities, how='left', on="Departure City")
data_cities = data_cities.rename(columns={"Departure City":"Arrival City", "Dep_lat":"Arr_lat", "Dep_lng":"Arr_lng", "Dep_population":"Arr_population", "Dep_density":"Arr_density"})
df_geo = df_geo.merge(data_cities, how='left', on="Arrival City")
df_geo.head()

,Departure City,Arrival City,Dep_lat,Dep_lng,Dep_population,Dep_density,Arr_lat,Arr_lng,Arr_population,Arr_density
0,Chicago,Chicago,41.8373,-87.6862,8604203,4574.0,41.8373,-87.6862,8604203,4574.0
1,Chicago,Las Vegas,41.8373,-87.6862,8604203,4574.0,36.2333,-115.2654,2104198,1773.0
2,Chicago,Denver,41.8373,-87.6862,8604203,4574.0,39.7621,-104.8759,2876625,1831.0
3,Chicago,Atlanta,41.8373,-87.6862,8604203,4574.0,33.7627,-84.4224,5449398,1441.0
4,Chicago,San Francisco,41.8373,-87.6862,8604203,4574.0,37.7562,-122.4430,3592294,7256.0


In [41]:
coord = df_geo.loc[:, ["Arr_lat", "Arr_lng", "Dep_lat", "Dep_lng"]].to_numpy()
lat1, lng1, lat2, lng2 = coord[:, 0], coord[:, 1], coord[:, 2], coord[:, 3]
distances = haversine_dist(lat1, lng1, lat2, lng2)
df_geo["Distance"] = distances
df_geo = df_geo.drop(columns={"Dep_lat", "Dep_lng", "Arr_lat", "Arr_lng"})

In [42]:
df_geo.head()

,Departure City,Arrival City,Dep_population,Dep_density,Arr_population,Arr_density,Distance
0,Chicago,Chicago,8604203,4574.0,8604203,4574.0,0.000
1,Chicago,Las Vegas,8604203,4574.0,2104198,1773.0,2450.532
2,Chicago,Denver,8604203,4574.0,2876625,1831.0,1462.665
3,Chicago,Atlanta,8604203,4574.0,5449398,1441.0,942.329
4,Chicago,San Francisco,8604203,4574.0,3592294,7256.0,2983.128


In [43]:
full_df = full_df.merge(df_geo, how='left', on=["Departure City", "Arrival City"])
full_df.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,log_PAX,std_wtd,year,month,day,weekday,...,Booking,Delta,Aairlines,Frequency,is_holiday,Dep_population,Dep_density,Arr_population,Arr_density,Distance
0,2012-06-19,ORD,DFW,12.875000,12.331296,9.812647,2012,6,19,1,...,703.650024,9.74,7.63,18.0,0,8604203,4574.0,5743938,1526.0,1285.250
1,2012-09-10,LAS,DEN,14.285714,10.775182,9.466734,2012,9,10,0,...,703.650024,9.74,7.63,14.0,0,2104198,1773.0,2876625,1831.0,990.537
2,2012-10-05,DEN,LAX,10.863636,11.083177,9.035883,2012,10,5,4,...,703.650024,9.74,7.63,16.0,0,2876625,1831.0,12750807,3276.0,1354.641
3,2011-10-09,ATL,ORD,11.480000,11.169268,7.990202,2011,10,9,6,...,526.010010,8.17,6.90,24.0,0,5449398,1441.0,8604203,4574.0,942.329
4,2012-02-21,DEN,SFO,11.450000,11.269364,9.517159,2012,2,21,1,...,526.010010,8.17,6.90,14.0,0,2876625,1831.0,3592294,7256.0,1536.901


# CSV Generation

This section handles the exportation of the final csv. The user must indicate which merging keys are going to be used to connect external_data.csv to the initial data contained in the testing dataset.

In [44]:
### CSV EXPORT PARAMETERS ###

merging_keys = ['DateOfDeparture', 'Departure', 'Arrival'] # keys to merge external_data.csv and the main data
ext_data_predictors = ["weekday", "Frequency", "Delta", "Booking", "Price", "is_holiday",
            "Arr_population", "Arr_density", "Distance"] # the columns which will be kept in external_data.csv
final_export_keys = merging_keys + ext_data_predictors

print(final_export_keys)

['DateOfDeparture', 'Departure', 'Arrival', 'Frequency', 'Delta', 'Booking', 'Price', 'is_holiday', 'Arr_population', 'Arr_density', 'Distance']


In [45]:
destination = os.path.join('submissions', 'cat', 'external_data.csv')
local_test = 'external_data.csv'

full_df = full_df.rename(columns={"Date":"DateOfDeparture"})
final_df = full_df.loc[:, final_export_keys]

final_df.info()
final_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11128 entries, 0 to 11127
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DateOfDeparture  11128 non-null  datetime64[ns]
 1   Departure        11128 non-null  object        
 2   Arrival          11128 non-null  object        
 3   Frequency        11128 non-null  float64       
 4   Delta            11128 non-null  float64       
 5   Booking          11128 non-null  float64       
 6   Price            11128 non-null  float64       
 7   is_holiday       11128 non-null  int64         
 8   Arr_population   11128 non-null  int64         
 9   Arr_density      11128 non-null  float64       
 10  Distance         11128 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(2), object(2)
memory usage: 1.0+ MB


,DateOfDeparture,Departure,Arrival,Frequency,Delta,Booking,Price,is_holiday,Arr_population,Arr_density,Distance
0,2012-06-19,ORD,DFW,18.0,9.74,703.650024,199.8,0,5743938,1526.0,1285.250
1,2012-09-10,LAS,DEN,14.0,9.74,703.650024,179.8,0,2876625,1831.0,990.537
2,2012-10-05,DEN,LAX,16.0,9.74,703.650024,183.4,0,12750807,3276.0,1354.641
3,2011-10-09,ATL,ORD,24.0,8.17,526.010010,177.9,0,8604203,4574.0,942.329
4,2012-02-21,DEN,SFO,14.0,8.17,526.010010,180.5,0,3592294,7256.0,1536.901


In [46]:
final_df.to_csv(destination, sep=",", index=False)
#final_df.to_csv(local_test, sep=",", index=False)